
# 📘 Image Processing & Analysis Toolkit 

This notebook explains, block-by-block, how the provided **Streamlit + OpenCV** application works, and demonstrates the core image-processing operations (color conversions, transformations, filtering, morphology, enhancement, and edge detection) on sample images.

> You can **run this notebook** to understand the pipeline and preview each operation locally. For the **GUI**, run the Streamlit app with:
```bash
python -m streamlit run app.py
```
(Replace `app.py` with your actual filename if different.)



## 1) Project Structure & Dependencies

**Key files** (suggested):
```
/your-project/
├── app.py                          # Streamlit GUI (your provided script)
├── requirements.txt                # streamlit, opencv-python, numpy, pillow, matplotlib
└── notebooks/
    └── Image_Processing_Toolkit_Explained.ipynb
```

**Install dependencies** (one-time):
```bash
pip install streamlit opencv-python numpy pillow matplotlib
```



## 2) High-Level Architecture

- **Input Layer**: Upload image via Streamlit (`st.file_uploader`) or capture frames from webcam.
- **Processing Core (OpenCV + NumPy)**:
  - Color conversions (RGB/BGR/HSV/YCbCr/Gray)
  - Geometric transforms (rotate/scale/translate/affine/perspective)
  - Filters (Gaussian/Median/Mean) and **Morphology** (dilate/erode/open/close)
  - Enhancement (histogram equalization, contrast stretch, sharpening)
  - Edge detection (Sobel, Laplacian, Canny)
- **Output Layer**: Side-by-side original vs processed preview, status bar (H, W, channels, size), **Save/Download** with compression options.


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# Utility to show images inline with Matplotlib
def show(img, title=None, cmap=None):
    plt.figure()
    if img.ndim == 2:
        plt.imshow(img, cmap=cmap if cmap is not None else 'gray')
    else:
        # Matplotlib expects RGB for correct colors
        if img.shape[2] == 3:
            plt.imshow(img)
        else:
            # If 4-channel, drop alpha for display
            plt.imshow(img[..., :3])
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()



## 3) Utility Functions (as in the app)

These functions mirror the logic in your Streamlit script. We'll re-define them here so we can run demonstrations without the GUI.


In [ ]:

def to_uint8(img):
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img

def to_pil(img_arr):
    img_arr = to_uint8(img_arr)
    return Image.fromarray(img_arr)

def convert_color(img, mode):
    # img expected as RGB for consistency in this notebook
    if mode == 'BGR':
        return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if mode == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if mode == 'YCbCr':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if mode == 'GRAY':
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img

def rotate_image(img, angle, center=None, scale=1.0):
    h, w = img.shape[:2]
    if center is None:
        center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, M, (w, h))

def scale_image(img, fx, fy):
    return cv2.resize(img, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)

def translate_image(img, tx, ty):
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    h, w = img.shape[:2]
    return cv2.warpAffine(img, M, (w, h))

def affine_transform(img, src_pts, dst_pts, dsize=None):
    M = cv2.getAffineTransform(np.float32(src_pts), np.float32(dst_pts))
    if dsize is None:
        h, w = img.shape[:2]
        dsize = (w, h)
    return cv2.warpAffine(img, M, dsize)

def perspective_transform(img, src_pts, dst_pts, dsize=None):
    M = cv2.getPerspectiveTransform(np.float32(src_pts), np.float32(dst_pts))
    if dsize is None:
        h, w = img.shape[:2]
        dsize = (w, h)
    return cv2.warpPerspective(img, M, dsize)

def apply_filter(img, method, ksize=3):
    if method == 'gaussian':
        return cv2.GaussianBlur(img, (ksize, ksize), 0)
    if method == 'median':
        return cv2.medianBlur(img, ksize)
    if method == 'mean':
        return cv2.blur(img, (ksize, ksize))
    return img

def morphological(img, op, ksize=3):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    if op == 'dilate':
        return cv2.dilate(img, kernel)
    if op == 'erode':
        return cv2.erode(img, kernel)
    if op == 'open':
        return cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    if op == 'close':
        return cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return img

def histogram_equalization(img):
    if img.ndim == 2:
        return cv2.equalizeHist(img)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)

def contrast_stretch(img, in_min=0, in_max=255):
    out = (img - in_min) * (255.0 / (in_max - in_min))
    return np.clip(out, 0, 255).astype(np.uint8)

def sharpen(img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(img, -1, kernel)

def sobel_edge(img):
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(gx ** 2 + gy ** 2)
    mag = np.uint8(np.clip(mag / (mag.max() + 1e-8) * 255, 0, 255))
    return mag

def laplacian_edge(img):
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lap = cv2.Laplacian(img, cv2.CV_64F)
    lap = np.uint8(np.clip(np.abs(lap) / (np.abs(lap).max() + 1e-8) * 255, 0, 255))
    return lap

def canny_edge(img, t1, t2):
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.Canny(img, t1, t2)

def save_image_bytes(img_arr, fmt='PNG', quality=95):
    pil = to_pil(img_arr)
    buf = BytesIO()
    save_kwargs = {}
    if fmt.upper() == 'JPEG':
        save_kwargs['format'] = 'JPEG'
        save_kwargs['quality'] = quality
    else:
        save_kwargs['format'] = fmt
    pil.save(buf, **save_kwargs)
    buf.seek(0)
    return buf



## 4) Load a Sample Image (or Generate One)

The Streamlit app loads user images. In this notebook, we'll either try to open an example image (if available) or generate a synthetic demo image (checkerboard + gradients) for demonstration.


In [ ]:

def synthetic_demo_image(h=256, w=256):
    # Create a simple synthetic RGB image with gradients and a checkerboard
    x = np.linspace(0, 255, w, dtype=np.uint8)
    y = np.linspace(0, 255, h, dtype=np.uint8)
    xv, yv = np.meshgrid(x, y)
    checker = (((xv // 32) % 2) ^ ((yv // 32) % 2)) * 255
    r = xv
    g = yv
    b = checker
    return np.dstack([r, g, b])

# Try to load an existing image from the environment (optional)
# Fallback to synthetic image
try_paths = [
    "/mnt/data/processed.png",
    "/mnt/data/Screenshot 2025-09-07 230156.png",
    "/mnt/data/Screenshot 2025-09-07 230218.png",
    "/mnt/data/Screenshot 2025-09-07 230412.png",
    "/mnt/data/Screenshot 2025-09-07 230447.png",
]

img = None
for p in try_paths:
    try:
        pil = Image.open(p).convert("RGB")
        img = np.array(pil)
        break
    except Exception:
        pass

if img is None:
    img = synthetic_demo_image()

show(img, "Original (RGB)")
img.shape



## 5) Color Conversions

The app supports: **RGB ↔ BGR**, **RGB ↔ HSV**, **RGB ↔ YCbCr**, **RGB ↔ Grayscale**.


In [ ]:

img_hsv = convert_color(img, 'HSV')
img_ycbcr = convert_color(img, 'YCbCr')
img_gray = convert_color(img, 'GRAY')

show(img_hsv, "HSV")
show(img_ycbcr, "YCbCr")
show(img_gray, "Grayscale", cmap='gray')



## 6) Geometric Transformations

Rotate, scale, translate, affine, and perspective transforms are used to modify spatial properties of the image.


In [ ]:

rot = rotate_image(img, angle=25)
scaled = scale_image(img, fx=0.6, fy=0.6)
trans = translate_image(img, tx=40, ty=30)

h, w = img.shape[:2]
src_aff = np.float32([[0, 0], [w - 1, 0], [0, h - 1]])
dst_aff = np.float32([[0, 0], [w - 1, 0], [int(0.2*w), int(0.4*h)]])
aff = affine_transform(img, src_aff, dst_aff)

src_p = np.float32([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]])
dst_p = np.float32([[0, int(0.1*h)], [w - 1, 0], [int(0.9*w), h - 1], [int(0.1*w), int(0.9*h)]])
persp = perspective_transform(img, src_p, dst_p)

show(rot, "Rotate +25°")
show(scaled, "Scale 0.6x")
show(trans, "Translate (40, 30)")
show(aff, "Affine Transform")
show(persp, "Perspective Transform")



## 7) Filtering & Morphology

- **Filtering**: Gaussian, Median, Mean (Average)  
- **Morphology**: Dilate, Erode, Open, Close (on binary/gray images)


In [ ]:

k = 5
gauss = apply_filter(img, 'gaussian', ksize=k)
median = apply_filter(img, 'median', ksize=k)
mean = apply_filter(img, 'mean', ksize=k)

show(gauss, f"Gaussian Blur (ksize={k})")
show(median, f"Median Blur (ksize={k})")
show(mean, f"Mean Blur (ksize={k})")

# Morphology works best on grayscale / binary
gray = convert_color(img, 'GRAY')
dil = morphological(gray, 'dilate', ksize=5)
ero = morphological(gray, 'erode', ksize=5)
opn = morphological(gray, 'open', ksize=5)
cls = morphological(gray, 'close', ksize=5)

show(dil, "Dilation (gray)", cmap='gray')
show(ero, "Erosion (gray)", cmap='gray')
show(opn, "Opening (gray)", cmap='gray')
show(cls, "Closing (gray)", cmap='gray')



## 8) Enhancement & Edge Detection

- **Enhancement**: Histogram Equalization, Contrast Stretching, Sharpening  
- **Edges**: Sobel, Laplacian, Canny


In [ ]:

eq = histogram_equalization(img)
cs = contrast_stretch(img, in_min=20, in_max=235)  # demo thresholds
sh = sharpen(img)

sob = sobel_edge(img)
lap = laplacian_edge(img)
can = canny_edge(img, 100, 200)

show(eq, "Histogram Equalization (Y channel)")
show(cs, "Contrast Stretch")
show(sh, "Sharpen")
show(sob, "Sobel Edge (magnitude)", cmap='gray')
show(lap, "Laplacian Edge", cmap='gray')
show(can, "Canny Edge", cmap='gray')



## 9) Saving with Compression (PNG / JPEG / BMP)

The app lets you save processed images with different formats and compare sizes. Below we simulate saving to bytes.


In [ ]:

buf_png = save_image_bytes(img, fmt='PNG')
buf_jpg = save_image_bytes(img, fmt='JPEG', quality=80)
buf_bmp = save_image_bytes(img, fmt='BMP')

sizes = {
    "PNG bytes": len(buf_png.getvalue()),
    "JPEG bytes (q=80)": len(buf_jpg.getvalue()),
    "BMP bytes": len(buf_bmp.getvalue())
}
sizes



## 10) How the Streamlit UI Maps to These Functions

- **Sidebar Inputs** (selectboxes, sliders, checkboxes) choose the operation and its parameters.  
- The app reads your file via `st.file_uploader`, then calls the corresponding functions (e.g., `rotate_image`, `apply_filter`, `canny_edge`).  
- **Display** uses `st.image` for **Original** (left) and **Processed** (right).  
- **Status Bar** shows dimensions, channels, format, file size.  
- **Save / Export** uses `BytesIO` to create downloadable images in your chosen format (PNG/JPEG/BMP).  
- **Webcam Mode**: A loop reads frames from `cv2.VideoCapture(0)`, converts to RGB, applies optional operations (e.g., Canny), and displays frames live.



## 11) Troubleshooting

- **`ModuleNotFoundError: No module named 'streamlit'`**  
  Install Streamlit in your active environment:  
  ```bash
  pip install streamlit
  ```

- **`streamlit` not recognized as a command (Windows)**  
  Run via the Python module to avoid PATH issues:  
  ```bash
  python -m streamlit run app.py
  ```

- **Webcam not opening**  
  - Make sure a webcam is available and not used by another app.  
  - Try index `1` instead of `0`: `cv2.VideoCapture(1)`.

- **Slow performance**  
  - Work with resized images during experiments.  
  - Prefer smaller kernel sizes and fewer chained operations.



## 12) Try-It-Yourself Exercises

1. Add **Bilateral Filter** to the filtering section and compare with Gaussian/Median.  
2. Add **Unsharp Masking** for sharpening with a tunable amount and radius.  
3. Build a **split-view overlay** using NumPy slicing (left = original, right = processed).  
4. Extend **webcam mode** to toggle multiple operations at once (e.g., sharpen + Canny).  
5. Add **bitwise operations** (AND/OR/XOR/NOT) with a synthetic mask to the app and test them.
